# Data

I have taken the data regarding toronto from the wikipedia page-"https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=935851093 and geosaptial data from this file-'http://cocl.us/Geospatial_data' .Then I merge the two dataframes obtained to get a cosolidated locational data of toronto.After that i use venues data from foursqaure api to obtain restraunt category information .The restarunt's are then categorized into cusine types like-afgani,indian etc-manually.After that i perform k-clustering to ascertain weather there are dominant cusine types in neighbourhoods and if there are then what are these.

# Data cleaning

In [1]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

In [2]:
req = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=935851093")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df = pd.read_html(str(table))


neighborhood=pd.DataFrame(df[0])

In [3]:
df1=neighborhood.dropna(subset = ["Borough"])
df1.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


In [4]:
df1 = df1.groupby('Postcode').agg({'Neighbourhood':'first', 
                             'Neighbourhood': ', '.join , 'Borough':'first'
                              }).reset_index()
df1.head()

,Postcode,Neighbourhood,Borough
0,M1A,Not assigned,Not assigned
1,M1B,"Rouge, Malvern",Scarborough
2,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
3,M1E,"Guildwood, Morningside, West Hill",Scarborough
4,M1G,Woburn,Scarborough


In [5]:
df1.Borough.fillna(df1.Neighbourhood,inplace=True)
df1.head()

,Postcode,Neighbourhood,Borough
0,M1A,Not assigned,Not assigned
1,M1B,"Rouge, Malvern",Scarborough
2,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
3,M1E,"Guildwood, Morningside, West Hill",Scarborough
4,M1G,Woburn,Scarborough


Above, i downloaded the postal code and neighbourhood information from wikipedia page. Then to clean the data i have performed foloowing actions-
1)remove rows having nan values in borugh column 
2)merging rows having same costal code

Downlaoding the geosaptial data and merging the two data frames

In [6]:
df = pd.read_csv('http://cocl.us/Geospatial_data')
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
dfinal = df.merge(df1, how='inner', left_on='Postal Code', right_on='Postcode')
dfinal.head(10)

,Postal Code,Latitude,Longitude,Postcode,Neighbourhood,Borough
0,M1B,43.806686,-79.194353,M1B,"Rouge, Malvern",Scarborough
1,M1C,43.784535,-79.160497,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,43.763573,-79.188711,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,43.770992,-79.216917,M1G,Woburn,Scarborough
4,M1H,43.773136,-79.239476,M1H,Cedarbrae,Scarborough
5,M1J,43.744734,-79.239476,M1J,Scarborough Village,Scarborough
6,M1K,43.727929,-79.262029,M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough
7,M1L,43.711112,-79.284577,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough
8,M1M,43.716316,-79.239476,M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough
9,M1N,43.692657,-79.264848,M1N,"Birch Cliff, Cliffside West",Scarborough


# Data analysis

Downlaoding relevant dependecies

In [8]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
from pprint import pprint 



from geopy.geocoders import Nominatim 



import matplotlib.cm as cm 
import matplotlib.colors as colors 

from pandas.io.json import json_normalize 
from collections import Counter 

from sklearn.cluster import KMeans 
! pip install folium
import folium
print('Libraries imported.')

     |████████████████████████████████| 92kB 7.1MB/s eta 0:00:011
Libraries imported.


Generating map of toronto with neighbourhoods superimposed on it.

In [9]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [10]:
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)



for lat, lng, borough, neighborhood in zip(dfinal['Latitude'], dfinal['Longitude'], dfinal['Borough'], dfinal['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough) 
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(Toronto_map)  
    
Toronto_map

# Fetching foursaquare venue category

In [11]:
CLIENT_ID = 'CE52W2D4UCAA0KJTKGZS2WKOC04CXC1551PPJJLPM5HXTMCX' 
CLIENT_SECRET = 'DQPSQALZQG5EOVAY1BKH3WF5C352RZB5QZM3FFFXCI02POUU'
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CE52W2D4UCAA0KJTKGZS2WKOC04CXC1551PPJJLPM5HXTMCX
CLIENT_SECRET:DQPSQALZQG5EOVAY1BKH3WF5C352RZB5QZM3FFFXCI02POUU


In [12]:
url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION)
venue_results = requests.get(url).json()

In [13]:
venue_list = venue_results['response']['categories']

In [14]:
len(venue_list)

10

In [15]:
for data in venue_list:
    print(data['id'], data['name'])

4d4b7104d754a06370d81259 Arts & Entertainment
4d4b7105d754a06372d81259 College & University
4d4b7105d754a06373d81259 Event
4d4b7105d754a06374d81259 Food
4d4b7105d754a06376d81259 Nightlife Spot
4d4b7105d754a06377d81259 Outdoors & Recreation
4d4b7105d754a06375d81259 Professional & Other Places
4e67e38e036454776db1fb3a Residence
4d4b7105d754a06378d81259 Shop & Service
4d4b7105d754a06379d81259 Travel & Transport


In [16]:
def flatten_Hierarchy(venue_list, checkParentID, category_dict, parent_id = ''):
    for data in venue_list:
        
        if checkParentID == True and data['id'] == parent_id:
            category_dict[data['id']] = data['name']
            flatten_Hierarchy(venue_list = data['categories'], checkParentID = False, category_dict = category_dict)
        
        elif checkParentID == False:
            category_dict[data['id']] = data['name']
            if len(data['categories']) != 0:
                flatten_Hierarchy(venue_list = data['categories'], checkParentID = False, category_dict = category_dict)
    
    return category_dict

In [17]:
category_dict = flatten_Hierarchy(venue_list, checkParentID=True, category_dict = {}, parent_id = '4d4b7105d754a06374d81259')

Exploring a neighbourhood to get a insight about what data is like

In [18]:
dfinal.loc[0, 'Neighbourhood']

'Rouge, Malvern'

In [ ]:
neighborhood_latitude = dfinal.loc[0, 'Latitude'] 
neighborhood_longitude = dfinal.loc[0, 'Longitude'] 

neighborhood_name = dfinal.loc[0, 'Neighbourhood'] 
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


In [ ]:
LIMIT = 1 
radius = 500 
categoryId = '4d4b7105d754a06374d81259' 



url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius,
    categoryId,
    LIMIT)
results = requests.get(url).json()
results['response']['venues']

[{'id': '4f5d15b7e4b06784f91bad71',
  'name': 'Second Cup',
  'location': {'address': '60 Grand Marshall Drive',
   'crossStreet': 'Sheppard',
   'lat': 43.802164535542744,
   'lng': -79.19611362644378,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.802164535542744,
     'lng': -79.19611362644378}],
   'distance': 522,
   'cc': 'CA',
   'country': 'Canada',
   'formattedAddress': ['60 Grand Marshall Drive (Sheppard)', 'Canada']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1586860004',
  'hasPerk': False}]

# Streamlining data for better analysis

The category of venue searched is indian restraunt,which is cusine type.
But there will be general names like-bakery,coffe shop etc.These names need to be removed for our analysis.
Also there are certain venue categories,whose frequency is very low,and should not be considered in determining the dominant food cusine.Therefore i have taken only those venue categories whose frequency is above 30.
Also there some  venue categories which are overlapping and hence it is best to place them under one category.For eg-sushi restraunt can be renamed as japanese restraunt.I have renamed such categories based on google searches and some might not be  hundered percent right but i believe that it was necessary given the large number of venue categories and their overlapping nature.

In [ ]:
def getNearbyFood(names, latitudes, longitudes, radius=1000, LIMIT=500):
    not_found = 0
    print('***Start ', end='')
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(' .', end='')
            
        
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            "4d4b7105d754a06374d81259", 
            LIMIT)
            
        try:
           
            results = requests.get(url).json()['response']['venues']
            
           
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['name'], 
                v['location']['lat'], 
                v['location']['lng'],  
                v['categories'][0]['name']) for v in results])
        except:
            not_found += 1


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print("\nDone*** with {} venues with incompelete information.".format(not_found))
    return(nearby_venues)

In [ ]:
import pickle # to serialize and deserialize a Python object structure
try:
    with open('toronto_food_venues.pkl', 'rb') as f:
        nyc_venues = pickle.load(f)
    print("---Dataframe Existed and Deserialized---")
except:
    toronto_venues = getNearbyFood(names=dfinal['Neighbourhood'],
                                        latitudes=dfinal['Latitude'],
                                        longitudes=dfinal['Longitude']
                                       )
    with open('toronto_food_venues.pkl', 'wb') as f:
        pickle.dump(toronto_venues, f)
    print("---Dataframe Created and Serialized---")

***Start  . . .

In [ ]:
toronto_venues.head()

In [ ]:
toronto_venues.groupby('Venue Category')['Venue Category'].count().sort_values(ascending=False)

In [ ]:
unique_categories = toronto_venues['Venue Category'].unique().tolist()
print(', '.join(str(x) for x in unique_categories))

In [ ]:
general=['Coffee Shop','Fast Food Restaurant','Fried Chicken Joint','Sandwich Place','Restaurant','Burger Joint','Bakery','Pizza Place',
         'Breakfast Spot','BBQ Join','Café','Fish & Chips Shop','Shawarma Place','Salad Place','Sports Bar','Grocery Store','Juice Bar','Snack Place',
         'Dessert Shop','Food Court','Halal Restaurant','Wings Joint','Food Truck','Ice Cream Shop','Vegetarian / Vegan Restaurant','Diner','Tea Room'
         ,'Gastropub','Bistro','Harbor / Marina','Hot Dog Joint','Chaat Place','Seafood Restaurant','Internet Cafe','Hotpot Restaurant','Bubble Tea Shop'
         ,'Noodle House','Hakka Restaurant','Hotpot Restaurant','Pastry Shop','Frozen Yogurt Shop','Chocolate Shop','Bar','Pub','Cafeteria','Buffet','Sake Bar'
         ,'Food & Drink Shop','Butcher','Gourmet Shop','Corporate Cafeteria','Pool Hall','Food','Event Space','Food Stand','Factory','Beach','Cupcake Shop'
         ,'Beer Bar','Brewery','Cocktail Bar','Supermarket','Comic Shop','Hardware Store','Smoothie Shop','Coworking Space','Pide Place','Food Service','Jazz Club'
         ,'Wine Bar','Creperie', 'Molecular Gastronomy Restaurant','Gay Bar','Shopping Mall','Theme Restaurant','Poke Place','Speakeasy',
         'Market','Convenience Store','Comfort Food Restaurant','Gluten-free Restaurant','Hotel Bar','Marijuana Dispensary','Hotel','Bike Shop','Pie Shop','Steakhouse'
        ,'Souvlaki Shop','Deli / Bodega','BBQ Joint','Bagel Shop','Office','Donut Shop','Irish Pub ','Empanada Restaurant','Mac & Cheese Joint']        

    

In [ ]:
counts=toronto_venues['Venue Category'].value_counts()
counts



In [ ]:
counts[counts >30]
df1=toronto_venues[toronto_venues['Venue Category'].isin(counts[counts > 30].index)]

In [ ]:
cusine_category=  list(set(unique_categories) - set(general))
print(', '.join(str(x) for x in cusine_category))

In [ ]:
df1 = df1[toronto_venues['Venue Category'].isin(cusine_category)]
df1.head(5)

In [ ]:
df1.groupby('Venue Category')['Venue Category'].count().sort_values(ascending=False)

In [ ]:
df1["Venue Category"].rename({'Sushi Restaurant':'Japanese Restaurant','Asian Restaurant':'Indian Restaurant'
                                        ,'Burrito Place':'Mexican Restaurant','Falafel Restaurant':'Middle Eastern Restaurant','Tapas Restaurant':'Spanish Restaurant',
                                        'Dumpling Restaurant':'Chinese Restaurant','Dim Sum Restaurant':'Cantonese Restaurant','New American Restaurant':'American Restaurant'
                                        ,'Cantonese Restaurant':'Cantonese Restaurant','Taco Place':'Mexican Restaurant','South Indian Restaurant':'Indian Restaurant'
                                        ,'Poutine Place':'American Restaurant','Cajun / Creole Restaurant':'French Restaurant ','Indian Chinese Restaurant':'Indian Restaurant'
                                        ,'Kebab Restaurant':'Pakistani Restaurant','Szechuan Restaurant':'Chinese Restaurant','Arepa Restaurant':'South American Restaurant',
                                        'Chinese Breakfast Place':'Chinese Restaurant','Yemeni Restaurant':'Middle Eastern Restaurant','Colombian Restaurant':'South American Restaurant'
                                        ,'Cuban Restaurant':'South American Restaurant','Udon Restaurant':'Japanese Restaurant'},inplace=True)


In [ ]:
df1.groupby('Venue Category').count()

In [ ]:
one_hot = pd.get_dummies(df1[['Venue Category']], prefix="", prefix_sep="")
one_hot['Neighborhood'] = df1['Neighborhood'] 


fixed_columns = [one_hot.columns[-1]] + list(one_hot.columns[:-1])
one_hot = one_hot[fixed_columns] 

one_hot.head(5)


In [ ]:
torronto_grouped = one_hot.groupby('Neighborhood').mean().reset_index()
torronto_grouped.head()

# Results

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']


columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [ ]:
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torronto_grouped['Neighborhood']

In [ ]:
for ind in np.arange(torronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
toronto_grouped_clustering = torronto_grouped.drop('Neighborhood', 1)

In [ ]:
sum_of_squared_distances = []
K = range(1,50)
for k in K:
    print(k, end=' ')
    kmeans = KMeans(n_clusters=k).fit(toronto_grouped_clustering)
    sum_of_squared_distances.append(kmeans.inertia_)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('sum_of_squared_distances')
plt.title('Elbow Method For Optimal k');

Elbow method is used to calaculate the optimal number of clusters.Optimal k is at the point where rate of the above graph changes abruplty ,which is 10 in this case.

In [ ]:
k = 10

toronto_grouped_clustering = torronto_grouped.drop('Neighborhood', 1)


kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)


kmeans.labels_[0:38]

In [ ]:
neighborhoods_venues_sorted.head(5)

In [ ]:
try:
    neighborhoods_venues_sorted.drop('Cluster Labels', axis=1)
except:
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
toronto_merged = neighborhoods_venues_sorted.join(dfinal.set_index('Neighbourhood'), on='Neighborhood')
toronto_merged.head()

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-10],
        fill=True,
        fill_color=rainbow[cluster-10],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
cluster_0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[1:12]]
cluster_0.head()

In [ ]:
cluster_1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[1:12]]
cluster_1.head(5)

In [ ]:
cluster_2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[1:12]]
cluster_2.head(5)

In [ ]:
cluster_3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[1:12]]
cluster_3.head(5)

In [ ]:
cluster_4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[1:12]]
cluster_4.head(5)

In [ ]:
cluster_5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[1:12]]
cluster_5.head(5)

In [ ]:
cluster_6 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[1:12]]
cluster_6.head(5)

In [ ]:
cluster_7 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[1:12]]
cluster_7.head(5)

In [ ]:
cluster_8 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[1:12]]
cluster_8.head(5)

In [ ]:
cluster_9 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[1:12]]
cluster_9.head(5)